# Creating Dataset for Predictive Model

## Predictor 1: Total Fatalities in Disney theme parks (By Year)

In [317]:
import pandas as pd
df_incidents = pd.read_csv("Disney Deadly Incidents - Total FL_CA-2.csv")

In [318]:
df_incidents.columns.unique()

Index(['Death Caused', 'Death How', 'Date of Incident', 'On Attraction',
       'Attraction Area', 'Location/Time', 'Age of Deceased',
       'Gender of Deceased', 'From Where', 'Fault',
       'Which State of Attraction'],
      dtype='object')

In [319]:
df_incidents['Date of Incident'] = pd.to_datetime(df_incidents['Date of Incident'], errors='coerce')
# Filtering for deaths after and including the year 2017
df_incidents_aft_2017 = df_incidents[df_incidents['Date of Incident'].dt.year >= 2017]
df_incidents_aft_2017
df_incidents_grouped = df_incidents_aft_2017.groupby('Date of Incident').size().reset_index(name='Total_Deaths')
df_incidents_grouped

Date of Incident  Total_Deaths
0        2017-02-01             1
1        2017-11-05             1
2        2018-06-10             1
3        2018-07-09             1
4        2018-08-18             1
5        2019-03-12             1
6        2019-08-29             1
7        2019-11-01             1
8        2020-03-04             1
9        2020-12-18             1
10       2021-03-20             1
11       2021-05-30             1
12       2021-08-22             1
13       2022-09-25             1
14       2022-12-03             1
15       2023-02-18             1
16       2023-05-22             1
17       2023-07-26             1

In [320]:
# Grouping fatalities by year
df_incidents_grouped['Date of Incident'] = pd.to_datetime(df_incidents_grouped['Date of Incident'])
df_yearly_deaths = (
    df_incidents_grouped.groupby(df_incidents_grouped['Date of Incident'].dt.year)['Total_Deaths']
    .sum()
    .reset_index()
    .rename(columns={'Date of Incident': 'Year'})
)
df_yearly_deaths

Year  Total_Deaths
0  2017             2
1  2018             3
2  2019             3
3  2020             2
4  2021             3
5  2022             2
6  2023             3

Above, we derive the yearly deaths on any Disney theme parks from 2017 to 2023.

## Main dataset

In [321]:
df_attraction_details = pd.read_csv("waiting_times.csv")
df_attraction_details.head(2)

WORK_DATE                 DEB_TIME  DEB_TIME_HOUR  \
0  2018-01-01  2018-01-01 21:00:00.000             21   
1  2018-01-01  2018-01-01 19:30:00.000             19   

                  FIN_TIME ENTITY_DESCRIPTION_SHORT  WAIT_TIME_MAX  NB_UNITS  \
0  2018-01-01 21:15:00.000           Roller Coaster              0       2.0   
1  2018-01-01 19:45:00.000              Bumper Cars              5      18.0   

   GUEST_CARRIED  CAPACITY  ADJUST_CAPACITY  OPEN_TIME  UP_TIME  DOWNTIME  \
0            0.0     0.000             0.00          0        0         0   
1          148.0   254.749           254.75         15       15         0   

   NB_MAX_UNIT  
0          2.0  
1         18.0

In [322]:
df_attraction_details['ENTITY_DESCRIPTION_SHORT'].unique()

array(['Roller Coaster', 'Bumper Cars', 'Rapids Ride', 'Crazy Dance',
       'Skyway', 'Free Fall', 'Monorail', 'Swing Ride', 'Crazy Bus',
       'Drop Tower', 'Spinning Coaster', 'Scooby Doo', 'Superman Ride',
       'Spiral Slide', 'Inverted Coaster', 'Water Ride', 'Power Tower',
       'Top Spin', 'Log Flume', 'Oz Theatre', 'Circus Train',
       'Giant Wheel', 'Kiddie Coaster', 'Bungee Jump', 'Zipline',
       'Aeroplane Ride', 'Haunted House', 'Reverse Bungee', 'Go-Karts',
       'Dizzy Dropper', 'Merry Go Round', 'Flying Coaster', 'Gondola',
       'Pirate Ship', 'Giga Coaster', 'Himalaya Ride', 'Sling Shot',
       'Vertical Drop', 'Tilt-A-Whirl'], dtype=object)

In [323]:
# Filtering for relevant variables for our dataset
df_attraction_details_filtered = df_attraction_details[['WORK_DATE', 'ENTITY_DESCRIPTION_SHORT','WAIT_TIME_MAX','NB_UNITS']]
df_attraction_details_filtered.head(5)

WORK_DATE ENTITY_DESCRIPTION_SHORT  WAIT_TIME_MAX  NB_UNITS
0  2018-01-01           Roller Coaster              0       2.0
1  2018-01-01              Bumper Cars              5      18.0
2  2018-01-01              Rapids Ride              0       1.0
3  2018-01-01              Crazy Dance              5       1.0
4  2018-01-01                   Skyway              5      15.0

Below, we derive a dataframe depicting the maximum wait time for a particular attraction and the total number of people which visited it within a given month and year.

In [324]:
df_attraction_details_filtered['WORK_DATE'] = pd.to_datetime(df_attraction_details_filtered['WORK_DATE'])
df_attraction_details_filtered['YEAR_MONTH'] = df_attraction_details_filtered['WORK_DATE'].dt.to_period('M')
df_result_attraction_details = df_attraction_details_filtered.groupby(['YEAR_MONTH', 'ENTITY_DESCRIPTION_SHORT']).agg(
    MAX_WAIT_TIME=('WAIT_TIME_MAX', 'max'),
    TOTAL_UNITS=('NB_UNITS', 'sum')
).reset_index()

df_result_attraction_details.rename(columns={'YEAR_MONTH': 'Year_Month', 'ENTITY_DESCRIPTION_SHORT': 'Attraction','MAX_WAIT_TIME': 'Max_wait_time','TOTAL_UNITS': 'Total_units' }, inplace=True)

df_result_attraction_details

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Year_Month      Attraction  Max_wait_time  Total_units
0       2018-01  Aeroplane Ride             80   33908.6666
1       2018-01     Bumper Cars             25   31192.0000
2       2018-01     Bungee Jump            300    6836.7335
3       2018-01    Circus Train             10    1736.0000
4       2018-01       Crazy Bus             95    6859.6004
...         ...             ...            ...          ...
2073    2022-08    Tilt-A-Whirl            220   15019.7331
2074    2022-08        Top Spin              0       0.0000
2075    2022-08   Vertical Drop             80    3572.8669
2076    2022-08      Water Ride             50    9050.6667
2077    2022-08         Zipline             80   12060.1333

[2078 rows x 4 columns]

Below, we get the total deaths which occured in any Disney theme park in the previous year, and merge that column with our main dataframe.

In [325]:
df_result_attraction_details['Year_Month'] = df_result_attraction_details['Year_Month'].astype(str)
df_result_attraction_details['Year_Month'] = pd.to_datetime(df_result_attraction_details['Year_Month'])
df_result_attraction_details['Year'] = df_result_attraction_details['Year_Month'].dt.year
df_yearly_deaths['Total_Deaths_Previous_Year'] = df_yearly_deaths['Total_Deaths'].shift(1, fill_value=0)
df_merged = df_result_attraction_details.merge(
    df_yearly_deaths[['Year', 'Total_Deaths_Previous_Year']], 
    left_on='Year', 
    right_on='Year', 
    how='left'
)
df_merged = df_merged.drop(columns=['Year'])
df_merged['Year_Month'] = df_merged['Year_Month'].dt.strftime('%Y-%m')
df_merged


Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Total_Deaths_Previous_Year  
0                              2  
1                              2  
2                              2  
3                              2  
4                              2  
...                          ...  
2073                           3  
2074                           3  
2075                           3  
2076                           3  
2077                           3  

[2078 rows x 5 columns]

## Predictor 2: Weather (Rainy or Non-Rainy days)

Below is a function which outputs weather features for any specified location (longitude and latitude) and period, which are useful in telling us about what the weather was like for the given period at that particular location.

In [326]:
from meteostat import Point, Daily

# Define location coordinates for Disneyland Orlando (appropriate for main dataset)
latitude = 28.3772
longitude = -81.5707
altitude = None 

start_date = '2018-01-01' 
end_date = '2022-08-31'  


start = pd.to_datetime(start_date)
end = pd.to_datetime(end_date)


location = Point(latitude, longitude, altitude)

# daily weather data
weather_data = Daily(location, start, end).fetch()
weather_data

tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2018-01-01  12.1   6.7  15.6   NaN   NaN   20.0  14.9   NaN  1021.7   NaN
2018-01-02   9.1   6.7  12.8   NaN   NaN    8.0  21.4   NaN  1024.4   NaN
2018-01-03   8.1   2.2  11.7   NaN   NaN  327.0  21.4   NaN  1017.0   NaN
2018-01-04   4.6  -1.7  11.7   NaN   NaN  293.0  14.3   NaN  1020.8   NaN
2018-01-05   7.0   1.7  12.8   0.0   NaN    NaN   9.9   NaN  1027.2   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2022-08-27  26.8  24.4  32.8   6.3   NaN    0.0   5.4   NaN  1015.0   NaN
2022-08-28  26.9  24.0  31.1  13.0   NaN   71.0   5.6   NaN  1014.8   NaN
2022-08-29  27.6  24.4  31.7   6.3   NaN  103.0   7.8   NaN  1015.4   NaN
2022-08-30  28.4  25.6  32.0   7.4   NaN  167.0  11.1   NaN  1015.6   NaN
2022-08-31  28.3  24.4  33.9   5.9   NaN   10.0   6.8   NaN  1015.6   NaN

[1704 rows x 10 columns]

In [327]:
# Categorizing the weather based on precipitation levels
weather_data = weather_data.fillna(0)
def categorize_weather(df):
    df["Not Rainy"] = (df["prcp"] == 0).astype(int)
    df["Rainy"] = (df["prcp"] > 0).astype(int)
    return df
weather_data = categorize_weather(weather_data)
weather_data

tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
time                                                                        
2018-01-01  12.1   6.7  15.6   0.0   0.0   20.0  14.9   0.0  1021.7   0.0   
2018-01-02   9.1   6.7  12.8   0.0   0.0    8.0  21.4   0.0  1024.4   0.0   
2018-01-03   8.1   2.2  11.7   0.0   0.0  327.0  21.4   0.0  1017.0   0.0   
2018-01-04   4.6  -1.7  11.7   0.0   0.0  293.0  14.3   0.0  1020.8   0.0   
2018-01-05   7.0   1.7  12.8   0.0   0.0    0.0   9.9   0.0  1027.2   0.0   
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...   
2022-08-27  26.8  24.4  32.8   6.3   0.0    0.0   5.4   0.0  1015.0   0.0   
2022-08-28  26.9  24.0  31.1  13.0   0.0   71.0   5.6   0.0  1014.8   0.0   
2022-08-29  27.6  24.4  31.7   6.3   0.0  103.0   7.8   0.0  1015.4   0.0   
2022-08-30  28.4  25.6  32.0   7.4   0.0  167.0  11.1   0.0  1015.6   0.0   
2022-08-31  28.3  24.4  33.9   5.9   0.0   10.0   6.8   0.0  1015.6   0.0   

            Not Rainy  Rainy  
time                          
2018-01-01          1      0  
2018-01-02          1      0  
2018-01-03          1      0  
2018-01-04          1      0  
2018-01-05          1      0  
...               ...    ...  
2022-08-27          0      1  
2022-08-28          0      1  
2022-08-29          0      1  
2022-08-30          0      1  
2022-08-31          0      1  

[1704 rows x 12 columns]

In [328]:
# Merging weather categories (Rainy or Non-rainy) to our main dataframe
df_merged2 = pd.merge(df_merged, monthly_weather_data, left_on="Year_Month", right_on="year_month", how="left")
df_merged2.drop(columns=["year_month"], inplace=True)
df_merged2

Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Total_Deaths_Previous_Year  Not Rainy     Rainy  
0                              2   0.935484  0.064516  
1                              2   0.935484  0.064516  
2                              2   0.935484  0.064516  
3                              2   0.935484  0.064516  
4                              2   0.935484  0.064516  
...                          ...        ...       ...  
2073                           3   0.032258  0.967742  
2074                           3   0.032258  0.967742  
2075                           3   0.032258  0.967742  
2076                           3   0.032258  0.967742  
2077                           3   0.032258  0.967742  

[2078 rows x 7 columns]

## Predictor 3: Natural Disasters 

In [329]:
df_disasters_FL = pd.read_csv("time-series-FL-cost-1980-2024.csv")
df_disasters_FL_filtered = df_disasters_FL[['Year','Drought Count', 'Flooding Count', 'Freeze Count', 'Severe Storm Count', 'Tropical Cyclone Count', 'Wildfire Count','Winter Storm Count', 'All Disasters Count' ]]

In [330]:
# Filtering for disaster data from 2018 to 2022, to align with records in our main dataframe
df_disasters_FL_filtered = df_disasters_FL_filtered[(df_disasters_FL_filtered["Year"] >= 2018) & 
                                       (df_disasters_FL_filtered["Year"] <= 2022)]
df_disasters_FL_filtered

Year  Drought Count  Flooding Count  Freeze Count  Severe Storm Count  \
38  2018              0               0             0                   2   
39  2019              0               0             0                   0   
40  2020              0               0             0                   4   
41  2021              0               0             0                   0   
42  2022              0               0             0                   2   

    Tropical Cyclone Count  Wildfire Count  Winter Storm Count  \
38                       1               0                   0   
39                       1               0                   0   
40                       3               0                   0   
41                       4               0                   0   
42                       2               0                   0   

    All Disasters Count  
38                    3  
39                    1  
40                    7  
41                    4  
42                    4

In [331]:
df_merged2["Year"] = df_merged2["Year_Month"].str[:4].astype(int)
df_merged2.rename(columns={"Total_Deaths_Previous_Year": "Park_Fatalities_Previous_Year", "Not Rainy": "Fraction of Not Rainy", "Rainy": "Fraction of Rainy"}, inplace=True)
df_merged3 = pd.merge(df_merged2, df_disasters_FL_filtered, on="Year", how="left")
df_merged3.drop(columns=["Year"], inplace=True)
for col in [
    "Drought Count", "Flooding Count", "Freeze Count",
    "Severe Storm Count", "Tropical Cyclone Count",
    "Wildfire Count", "Winter Storm Count", "All Disasters Count"
]:
    df_merged3[col] = df_merged3[col] / 12 # Normalize yearly disaster counts by 12
df_merged3

Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Park_Fatalities_Previous_Year  Fraction of Not Rainy  Fraction of Rainy  \
0                                 2               0.935484           0.064516   
1                                 2               0.935484           0.064516   
2                                 2               0.935484           0.064516   
3                                 2               0.935484           0.064516   
4                                 2               0.935484           0.064516   
...                             ...                    ...                ...   
2073                              3               0.032258           0.967742   
2074                              3               0.032258           0.967742   
2075                              3               0.032258           0.967742   
2076                              3               0.032258           0.967742   
2077                              3               0.032258           0.967742   

      Drought Count  Flooding Count  Freeze Count  Severe Storm Count  \
0               0.0             0.0           0.0            0.166667   
1               0.0             0.0           0.0            0.166667   
2               0.0             0.0           0.0            0.166667   
3               0.0             0.0           0.0            0.166667   
4               0.0             0.0           0.0            0.166667   
...             ...             ...           ...                 ...   
2073            0.0             0.0           0.0            0.166667   
2074            0.0             0.0           0.0            0.166667   
2075            0.0             0.0           0.0            0.166667   
2076            0.0             0.0           0.0            0.166667   
2077            0.0             0.0           0.0            0.166667   

      Tropical Cyclone Count  Wildfire Count  Winter Storm Count  \
0                   0.083333             0.0                 0.0   
1                   0.083333             0.0                 0.0   
2                   0.083333             0.0                 0.0   
3                   0.083333             0.0                 0.0   
4                   0.083333             0.0                 0.0   
...                      ...             ...                 ...   
2073                0.166667             0.0                 0.0   
2074                0.166667             0.0                 0.0   
2075                0.166667             0.0                 0.0   
2076                0.166667             0.0                 0.0   
2077                0.166667             0.0                 0.0   

      All Disasters Count  
0                0.250000  
1                0.250000  
2                0.250000  
3                0.250000  
4                0.250000  
...                   ...  
2073             0.333333  
2074             0.333333  
2075             0.333333  
2076             0.333333  
2077             0.333333  

[2078 rows x 15 columns]

Above, we get the resulting dataframe after merging records of different natural disasters (normalized by 12) to our main dataframe.

## Predictor 4: Number of public holidays in each month 

In [332]:
# Defining public holidays in Florida
public_holidays = {
    'January': ['New Year’s Day', 'Martin Luther King Jr. Day'],
    'February': ['Presidents’ Day'],
    'May': ['Memorial Day'],
    'June': ['Juneteenth'],
    'July': ['Independence Day'],
    'September': ['Labor Day'],
    'November': ['Veterans’ Day', 'Thanksgiving Day', 'Day after Thanksgiving'],
    'December': ['Christmas Day']
}

holidays_per_month = {month: len(holidays) for month, holidays in public_holidays.items()}


def count_holidays(year_month):
    month_number = int(year_month.split('-')[1])
    month_name = pd.to_datetime(f'2021-{month_number:02d}-01').strftime('%B')
    return holidays_per_month.get(month_name, 0)

df_merged3['Public_Holidays_in_Month'] = df_merged3['Year_Month'].apply(count_holidays)
df_merged3

Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Park_Fatalities_Previous_Year  Fraction of Not Rainy  Fraction of Rainy  \
0                                 2               0.935484           0.064516   
1                                 2               0.935484           0.064516   
2                                 2               0.935484           0.064516   
3                                 2               0.935484           0.064516   
4                                 2               0.935484           0.064516   
...                             ...                    ...                ...   
2073                              3               0.032258           0.967742   
2074                              3               0.032258           0.967742   
2075                              3               0.032258           0.967742   
2076                              3               0.032258           0.967742   
2077                              3               0.032258           0.967742   

      Drought Count  Flooding Count  Freeze Count  Severe Storm Count  \
0               0.0             0.0           0.0            0.166667   
1               0.0             0.0           0.0            0.166667   
2               0.0             0.0           0.0            0.166667   
3               0.0             0.0           0.0            0.166667   
4               0.0             0.0           0.0            0.166667   
...             ...             ...           ...                 ...   
2073            0.0             0.0           0.0            0.166667   
2074            0.0             0.0           0.0            0.166667   
2075            0.0             0.0           0.0            0.166667   
2076            0.0             0.0           0.0            0.166667   
2077            0.0             0.0           0.0            0.166667   

      Tropical Cyclone Count  Wildfire Count  Winter Storm Count  \
0                   0.083333             0.0                 0.0   
1                   0.083333             0.0                 0.0   
2                   0.083333             0.0                 0.0   
3                   0.083333             0.0                 0.0   
4                   0.083333             0.0                 0.0   
...                      ...             ...                 ...   
2073                0.166667             0.0                 0.0   
2074                0.166667             0.0                 0.0   
2075                0.166667             0.0                 0.0   
2076                0.166667             0.0                 0.0   
2077                0.166667             0.0                 0.0   

      All Disasters Count  Public_Holidays_in_Month  
0                0.250000                         2  
1                0.250000                         2  
2                0.250000                         2  
3                0.250000                         2  
4                0.250000                         2  
...                   ...                       ...  
2073             0.333333                         0  
2074             0.333333                         0  
2075             0.333333                         0  
2076             0.333333                

## Predictor 5: Different seasons 

In [333]:
def determine_season(year_month):
    month = int(year_month.split('-')[1])
    if month in [3, 4, 5]:   # Spring: March - May
        return 'Spring'
    elif month in [6, 7, 8]:  # Summer: June - August
        return 'Summer'
    elif month in [9, 10, 11]: # Autumn: September - November
        return 'Autumn'
    else:                     # Winter: December - February
        return 'Winter'


df_merged3['Season'] = df_merged3['Year_Month'].apply(determine_season)
df_merged3['Spring'] = (df_merged3['Season'] == 'Spring').astype(int)
df_merged3['Summer'] = (df_merged3['Season'] == 'Summer').astype(int)
df_merged3['Autumn'] = (df_merged3['Season'] == 'Autumn').astype(int)
df_merged3['Winter'] = (df_merged3['Season'] == 'Winter').astype(int)

df_merged3.drop(columns=['Season'], inplace=True)
df_merged3

Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Park_Fatalities_Previous_Year  Fraction of Not Rainy  Fraction of Rainy  \
0                                 2               0.935484           0.064516   
1                                 2               0.935484           0.064516   
2                                 2               0.935484           0.064516   
3                                 2               0.935484           0.064516   
4                                 2               0.935484           0.064516   
...                             ...                    ...                ...   
2073                              3               0.032258           0.967742   
2074                              3               0.032258           0.967742   
2075                              3               0.032258           0.967742   
2076                              3               0.032258           0.967742   
2077                              3               0.032258           0.967742   

      Drought Count  Flooding Count  Freeze Count  Severe Storm Count  \
0               0.0             0.0           0.0            0.166667   
1               0.0             0.0           0.0            0.166667   
2               0.0             0.0           0.0            0.166667   
3               0.0             0.0           0.0            0.166667   
4               0.0             0.0           0.0            0.166667   
...             ...             ...           ...                 ...   
2073            0.0             0.0           0.0            0.166667   
2074            0.0             0.0           0.0            0.166667   
2075            0.0             0.0           0.0            0.166667   
2076            0.0             0.0           0.0            0.166667   
2077            0.0             0.0           0.0            0.166667   

      Tropical Cyclone Count  Wildfire Count  Winter Storm Count  \
0                   0.083333             0.0                 0.0   
1                   0.083333             0.0                 0.0   
2                   0.083333             0.0                 0.0   
3                   0.083333             0.0                 0.0   
4                   0.083333             0.0                 0.0   
...                      ...             ...                 ...   
2073                0.166667             0.0                 0.0   
2074                0.166667             0.0                 0.0   
2075                0.166667             0.0                 0.0   
2076                0.166667             0.0                 0.0   
2077                0.166667             0.0                 0.0   

      All Disasters Count  Public_Holidays_in_Month  Spring  Summer  Autumn  \
0                0.250000                         2       0       0       0   
1                0.250000                         2       0       0       0   
2                0.250000                         2       0       0       0   
3                0.250000                         2       0       0       0   
4                0.250000                         2       0       0       0   
...                   ...                       ...     ...     ...     ...   
2073             0.333333   

## Predictor 6: Ongoing disasters (By month)

In [351]:
df_disasters_FL2 = pd.read_csv("events-FL-1980-2024.csv")
df_disasters_FL2 = df_disasters_FL2[['Name','Disaster','Begin Date','End Date']]
df_disasters_FL2

Name          Disaster  \
0                       Florida Freeze (January 1981)            Freeze   
1   Midwest/Southeast/Northeast Winter Storm, Cold...      Winter Storm   
2   Gulf States Storms and Flooding (December 1982...          Flooding   
3                    Freeze/Cold Wave (December 1983)            Freeze   
4      Tornadoes, Severe Storms, Floods (Spring 1984)      Severe Storm   
..                                                ...               ...   
89                        Southern Derecho (May 2024)      Severe Storm   
90                      Hurricane Debby (August 2024)  Tropical Cyclone   
91                Hurricane Francine (September 2024)  Tropical Cyclone   
92                  Hurricane Helene (September 2024)  Tropical Cyclone   
93                    Hurricane Milton (October 2024)  Tropical Cyclone   

    Begin Date  End Date  
0     19810112  19810114  
1     19820108  19820116  
2     19821201  19830115  
3     19831215  19831225  
4     19840327  19840407  
..         ...       ...  
89    20240516  20240517  
90    20240805  20240809  
91    20240911  20240912  
92    20240924  20240929  
93    20241009  20241010  

[94 rows x 4 columns]

In [335]:
df_disasters_FL2["Begin Date"] = df_disasters_FL2["Begin Date"].astype(int)
df_disasters_FL2["End Date"] = df_disasters_FL2["End Date"].astype(int)

start_date = 20180101  # January 1, 2018
end_date = 20220831    # August 31, 2022

df_disasters_FL2_filtered = df_disasters_FL2[(df_disasters_FL2["End Date"] >= start_date) & 
                                        (df_disasters_FL2["Begin Date"] <= end_date)]
df_disasters_FL2_filtered

Name          Disaster  \
56  Southeastern Tornadoes and Severe Weather (Mar...      Severe Storm   
57  Southern and Eastern Tornadoes and Severe Weat...      Severe Storm   
58                   Hurricane Michael (October 2018)  Tropical Cyclone   
59                  Hurricane Dorian (September 2019)  Tropical Cyclone   
60  South, East and Northeast Severe Weather (Febr...      Severe Storm   
61  Southeast and Eastern Tornado Outbreak (April ...      Severe Storm   
62               Southern Severe Weather (April 2020)      Severe Storm   
63  South, Central and Eastern Severe Weather (May...      Severe Storm   
64                     Hurricane Isaias (August 2020)  Tropical Cyclone   
65                   Hurricane Sally (September 2020)  Tropical Cyclone   
66                 Tropical Storm Eta (November 2020)  Tropical Cyclone   
67                    Tropical Storm Elsa (July 2021)  Tropical Cyclone   
68                  Tropical Storm Fred (August 2021)  Tropical Cyclone   
69                        Hurricane Ida (August 2021)  Tropical Cyclone   
70                Hurricane Nicholas (September 2021)  Tropical Cyclone   
71             Southern Tornado Outbreak (March 2022)      Severe Storm   
72            Southeast Tornado Outbreak (April 2022)      Severe Storm   

    Begin Date  End Date  
56    20180318  20180321  
57    20180413  20180416  
58    20181010  20181011  
59    20190828  20190906  
60    20200205  20200207  
61    20200412  20200413  
62    20200421  20200423  
63    20200520  20200523  
64    20200803  20200804  
65    20200915  20200917  
66    20201108  20201112  
67    20210707  20210709  
68    20210816  20210818  
69    20210829  20210901  
70    20210914  20210918  
71    20220330  20220330  
72    20220404  20220406

Below, we merge records of the number of ongoing disasters happening in each given month and year.

In [336]:
df_merged3["Year_Month_dt"] = pd.to_datetime(df_merged3["Year_Month"], format="%Y-%m")
df_disasters_FL2_filtered["Begin Date"] = pd.to_datetime(df_disasters_FL2_filtered["Begin Date"], format="%Y%m%d")
df_disasters_FL2_filtered["End Date"] = pd.to_datetime(df_disasters_FL2_filtered["End Date"], format="%Y%m%d")

def count_ongoing_disasters(year_month):
    month_start = year_month  
    month_end = year_month + pd.DateOffset(days=30)  

    return ((df_disasters_FL2_filtered["Begin Date"] <= month_end) & 
            (df_disasters_FL2_filtered["End Date"] >= month_start)).sum()

df_merged3["Ongoing Disasters"] = df_merged3["Year_Month_dt"].apply(count_ongoing_disasters)
df_merged3.drop(columns=["Year_Month_dt"], inplace=True)
df_merged3

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Park_Fatalities_Previous_Year  Fraction of Not Rainy  Fraction of Rainy  \
0                                 2               0.935484           0.064516   
1                                 2               0.935484           0.064516   
2                                 2               0.935484           0.064516   
3                                 2               0.935484           0.064516   
4                                 2               0.935484           0.064516   
...                             ...                    ...                ...   
2073                              3               0.032258           0.967742   
2074                              3               0.032258           0.967742   
2075                              3               0.032258           0.967742   
2076                              3               0.032258           0.967742   
2077                              3               0.032258           0.967742   

      Drought Count  Flooding Count  Freeze Count  ...  \
0               0.0             0.0           0.0  ...   
1               0.0             0.0           0.0  ...   
2               0.0             0.0           0.0  ...   
3               0.0             0.0           0.0  ...   
4               0.0             0.0           0.0  ...   
...             ...             ...           ...  ...   
2073            0.0             0.0           0.0  ...   
2074            0.0             0.0           0.0  ...   
2075            0.0             0.0           0.0  ...   
2076            0.0             0.0           0.0  ...   
2077            0.0             0.0           0.0  ...   

      Tropical Cyclone Count  Wildfire Count  Winter Storm Count  \
0                   0.083333             0.0                 0.0   
1                   0.083333             0.0                 0.0   
2                   0.083333             0.0                 0.0   
3                   0.083333             0.0                 0.0   
4                   0.083333             0.0                 0.0   
...                      ...             ...                 ...   
2073                0.166667             0.0                 0.0   
2074                0.166667             0.0                 0.0   
2075                0.166667             0.0                 0.0   
2076                0.166667             0.0                 0.0   
2077                0.166667             0.0                 0.0   

      All Disasters Count  Public_Holidays_in_Month  Spring  Summer  Autumn  \
0                0.250000                         2       0       0       0   
1                0.250000                         2       0       0       0   
2                0.250000                         2       0       0       0   
3                0.250000                         2       0       0       0   
4                0.250000                         2       0       0       0   
...                   ...                       ...     ...     ...     ...   
2073             0.333333                         0       0       1       0   
2074             0.333333                         0       0       1       0   
2075             0.333333                         

## Predictor 7: Average number of night shows (Disney event) in each month and year

In [337]:
df_parade_show = pd.read_excel("parade_night_show.xlsx")
df_parade_show = df_parade_show[['WORK_DATE','PARADE_1','PARADE_2']]
df_parade_show

WORK_DATE  PARADE_1  PARADE_2
0   2018-10-01  17:30:00  12:10:00
1   2018-10-02  17:30:00  12:10:00
2   2018-10-03  17:30:00  12:10:00
3   2018-10-04  17:30:00  12:10:00
4   2018-10-05  17:30:00  12:10:00
..         ...       ...       ...
666 2022-08-14  17:30:00       NaN
667 2022-08-15  17:30:00       NaN
668 2022-08-16  17:30:00       NaN
669 2022-08-17  17:30:00       NaN
670 2022-08-18  17:30:00       NaN

[671 rows x 3 columns]

In [338]:
df_parade_show["Parade_Count"] = df_parade_show[["PARADE_1", "PARADE_2"]].notna().sum(axis=1)
df_parade_show["Parade_Count"] = df_parade_show["Parade_Count"].replace(0, 0)
df_parade_show=df_parade_show[['WORK_DATE','Parade_Count']]
df_parade_show

WORK_DATE  Parade_Count
0   2018-10-01             2
1   2018-10-02             2
2   2018-10-03             2
3   2018-10-04             2
4   2018-10-05             2
..         ...           ...
666 2022-08-14             1
667 2022-08-15             1
668 2022-08-16             1
669 2022-08-17             1
670 2022-08-18             1

[671 rows x 2 columns]

In [339]:
df_parade_show["Year_Month"] = df_parade_show["WORK_DATE"].dt.to_period("M").astype(str)

# Average number of parades per month
df_avg_parades = df_parade_show.groupby("Year_Month")["Parade_Count"].mean().reset_index()
df_avg_parades.rename(columns={"Parade_Count": "Avg_no_parades"}, inplace=True)
df_avg_parades

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Year_Month  Avg_no_parades
0     2018-10        2.000000
1     2018-11        1.833333
2     2018-12        2.000000
3     2019-01        1.193548
4     2019-02        1.000000
5     2019-03        1.000000
6     2019-04        1.000000
7     2019-05        1.000000
8     2019-06        1.000000
9     2019-07        1.000000
10    2019-08        1.000000
11    2019-09        1.100000
12    2019-10        2.000000
13    2019-11        1.866667
14    2019-12        2.000000
15    2020-01        1.193548
16    2020-02        1.000000
17    2020-03        1.000000
18    2022-04        1.000000
19    2022-05        1.000000
20    2022-06        1.000000
21    2022-07        1.000000
22    2022-08        1.000000

Below, as data from Jan 2018 to Sept 2018 is missing, we impute the average number of night shows for that period to be 2, since the average number of night shows in October, November and December are all close to or exactly 2. Next, research tells us that for the missing data from April 2020 to March 2022, there were no night shows occuring due to COVID measures during that period, hence we set those values to be 0.

In [340]:
default_parades = {}

# Setting Avg_no_parades = 2 for months before 2018-10
for year in range(2018):
    for month in range(1, 13):
        key = f"{year}-{month:02d}"
        default_parades[key] = 2.0

for month in range(1, 10):  # Jan 2018 to Sep 2018
    key = f"2018-{month:02d}"
    default_parades[key] = 2.0

# Setting Avg_no_parades = 0 for Covid-affected months (Apr 2020 - Mar 2022)
for year in [2020, 2021, 2022]:
    for month in range(1, 13):
        key = f"{year}-{month:02d}"
        if year == 2020 and month >= 4 or year in [2021] or (year == 2022 and month <= 3):
            default_parades[key] = 0.0

parades_dict = df_avg_parades.set_index("Year_Month")["Avg_no_parades"].to_dict()
parades_dict.update(default_parades)
df_merged3["Avg_no_parades"] = df_merged3["Year_Month"].map(parades_dict)
df_merged3

Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Park_Fatalities_Previous_Year  Fraction of Not Rainy  Fraction of Rainy  \
0                                 2               0.935484           0.064516   
1                                 2               0.935484           0.064516   
2                                 2               0.935484           0.064516   
3                                 2               0.935484           0.064516   
4                                 2               0.935484           0.064516   
...                             ...                    ...                ...   
2073                              3               0.032258           0.967742   
2074                              3               0.032258           0.967742   
2075                              3               0.032258           0.967742   
2076                              3               0.032258           0.967742   
2077                              3               0.032258           0.967742   

      Drought Count  Flooding Count  Freeze Count  ...  Wildfire Count  \
0               0.0             0.0           0.0  ...             0.0   
1               0.0             0.0           0.0  ...             0.0   
2               0.0             0.0           0.0  ...             0.0   
3               0.0             0.0           0.0  ...             0.0   
4               0.0             0.0           0.0  ...             0.0   
...             ...             ...           ...  ...             ...   
2073            0.0             0.0           0.0  ...             0.0   
2074            0.0             0.0           0.0  ...             0.0   
2075            0.0             0.0           0.0  ...             0.0   
2076            0.0             0.0           0.0  ...             0.0   
2077            0.0             0.0           0.0  ...             0.0   

      Winter Storm Count  All Disasters Count  Public_Holidays_in_Month  \
0                    0.0             0.250000                         2   
1                    0.0             0.250000                         2   
2                    0.0             0.250000                         2   
3                    0.0             0.250000                         2   
4                    0.0             0.250000                         2   
...                  ...                  ...                       ...   
2073                 0.0             0.333333                         0   
2074                 0.0             0.333333                         0   
2075                 0.0             0.333333                         0   
2076                 0.0             0.333333                         0   
2077                 0.0             0.333333                         0   

      Spring  Summer  Autumn  Winter  Ongoing Disasters  Avg_no_parades  
0          0       0       0       1                  0             2.0  
1          0       0       0       1                  0             2.0  
2          0       0       0       1                  0             2.0  
3          0       0       0       1                  0             2.0  
4          0       0       0       1                  0             2.0  
...      ...     ...     ...     ...     

Above, we merged the average number of night shows with our main dataframe.

## Predictor 8: Market Competition from another theme park within close proximity (SeaWorld Orlando)

In [341]:
df_attendee_parks = pd.read_csv("attendee.csv")
df_attendee_parks

name  year  attendee_count
0           Alton Towers  2023         2350000
1           Alton Towers  2022         2300000
2           Alton Towers  2021         1800000
3           Alton Towers  2020          670000
4           Alton Towers  2019         2520000
...                  ...   ...             ...
1406  Beto Carrero World  2016         2080000
1407  Beto Carrero World  2015         2000000
1408  Beto Carrero World  2014         1818000
1409  Beto Carrero World  2013         1530000
1410  Beto Carrero World  2012         1500000

[1411 rows x 3 columns]

In [342]:
# Filtering for records pertaining to SeaWorld Orlando and from 2018 to 2022
df_seaworld_orlando = df_attendee_parks[df_attendee_parks["name"] == "Seaworld Orlando"]
df_seaworld_orlando['Average_attendee_seaworld'] = df_seaworld_orlando['attendee_count']/12 # Data is yearly, so we derive average attendees per month
df_seaworld_orlando_filtered = df_seaworld_orlando[['year','Average_attendee_seaworld']]
df_seaworld_orlando_filtered = df_seaworld_orlando_filtered[
    (df_seaworld_orlando_filtered["year"] >= 2018) & 
    (df_seaworld_orlando_filtered["year"] <= 2022)
]
df_seaworld_orlando_filtered

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


year  Average_attendee_seaworld
355  2022              371166.666667
356  2021              254250.000000
357  2020              133166.666667
358  2019              386666.666667
359  2018              382833.333333

In [343]:
df_merged3["Year"] = df_merged3["Year_Month"].str[:4].astype(int)
df_merged3 = df_merged3.merge(df_seaworld_orlando_filtered, left_on="Year", right_on="year", how="left")
df_merged3.drop(columns=["year"], inplace=True)
df_merged3.drop(columns=["Year"], inplace=True)


In [344]:
df_merged3.columns.unique()

Index(['Year_Month', 'Attraction', 'Max_wait_time', 'Total_units',
       'Park_Fatalities_Previous_Year', 'Fraction of Not Rainy',
       'Fraction of Rainy', 'Drought Count', 'Flooding Count', 'Freeze Count',
       'Severe Storm Count', 'Tropical Cyclone Count', 'Wildfire Count',
       'Winter Storm Count', 'All Disasters Count', 'Public_Holidays_in_Month',
       'Spring', 'Summer', 'Autumn', 'Winter', 'Ongoing Disasters',
       'Avg_no_parades', 'Average_attendee_seaworld'],
      dtype='object')

In [345]:
df_merged3

Year_Month      Attraction  Max_wait_time  Total_units  \
0       2018-01  Aeroplane Ride             80   33908.6666   
1       2018-01     Bumper Cars             25   31192.0000   
2       2018-01     Bungee Jump            300    6836.7335   
3       2018-01    Circus Train             10    1736.0000   
4       2018-01       Crazy Bus             95    6859.6004   
...         ...             ...            ...          ...   
2073    2022-08    Tilt-A-Whirl            220   15019.7331   
2074    2022-08        Top Spin              0       0.0000   
2075    2022-08   Vertical Drop             80    3572.8669   
2076    2022-08      Water Ride             50    9050.6667   
2077    2022-08         Zipline             80   12060.1333   

      Park_Fatalities_Previous_Year  Fraction of Not Rainy  Fraction of Rainy  \
0                                 2               0.935484           0.064516   
1                                 2               0.935484           0.064516   
2                                 2               0.935484           0.064516   
3                                 2               0.935484           0.064516   
4                                 2               0.935484           0.064516   
...                             ...                    ...                ...   
2073                              3               0.032258           0.967742   
2074                              3               0.032258           0.967742   
2075                              3               0.032258           0.967742   
2076                              3               0.032258           0.967742   
2077                              3               0.032258           0.967742   

      Drought Count  Flooding Count  Freeze Count  ...  Winter Storm Count  \
0               0.0             0.0           0.0  ...                 0.0   
1               0.0             0.0           0.0  ...                 0.0   
2               0.0             0.0           0.0  ...                 0.0   
3               0.0             0.0           0.0  ...                 0.0   
4               0.0             0.0           0.0  ...                 0.0   
...             ...             ...           ...  ...                 ...   
2073            0.0             0.0           0.0  ...                 0.0   
2074            0.0             0.0           0.0  ...                 0.0   
2075            0.0             0.0           0.0  ...                 0.0   
2076            0.0             0.0           0.0  ...                 0.0   
2077            0.0             0.0           0.0  ...                 0.0   

      All Disasters Count  Public_Holidays_in_Month  Spring  Summer  Autumn  \
0                0.250000                         2       0       0       0   
1                0.250000                         2       0       0       0   
2                0.250000                         2       0       0       0   
3                0.250000                         2       0       0       0   
4                0.250000                         2       0       0       0   
...                   ...                       ...     ...     ...     ...   
2073             0.333333                         0       0       1       0   
2074             0.333333                         0       0       1       0   
2075             0.333333                         0       0       1       0   
2076             0.333333                         0       0       1       0   
2077             0.333333                         0       0       1       0   

      Winter  Ongoing Disasters  Avg_no_parades  Average_attendee_seaworld  
0          1                  0             2.0              382833.333333  
1          1                  0             2.0              382833.333333  
2          1                  0             2.0              382833.333333  
3          1                  0             2.0              382833.333333  
4   

Above, we finalize our merged dataframe with all predictors included.

Below, we perform one-hot encoding for our categorical variable "Attraction" and derive our final dataset, ready to use in the model selection and training procedure.

In [349]:
df_final = pd.get_dummies(df_merged3, columns=["Attraction"], drop_first=True)
attraction_columns = [col for col in df_final.columns if col.startswith("Attraction_")]
df_final[attraction_columns] = df_final[attraction_columns].astype(int)
df_final

Year_Month  Max_wait_time  Total_units  Park_Fatalities_Previous_Year  \
0       2018-01             80   33908.6666                              2   
1       2018-01             25   31192.0000                              2   
2       2018-01            300    6836.7335                              2   
3       2018-01             10    1736.0000                              2   
4       2018-01             95    6859.6004                              2   
...         ...            ...          ...                            ...   
2073    2022-08            220   15019.7331                              3   
2074    2022-08              0       0.0000                              3   
2075    2022-08             80    3572.8669                              3   
2076    2022-08             50    9050.6667                              3   
2077    2022-08             80   12060.1333                              3   

      Fraction of Not Rainy  Fraction of Rainy  Drought Count  Flooding Count  \
0                  0.935484           0.064516            0.0             0.0   
1                  0.935484           0.064516            0.0             0.0   
2                  0.935484           0.064516            0.0             0.0   
3                  0.935484           0.064516            0.0             0.0   
4                  0.935484           0.064516            0.0             0.0   
...                     ...                ...            ...             ...   
2073               0.032258           0.967742            0.0             0.0   
2074               0.032258           0.967742            0.0             0.0   
2075               0.032258           0.967742            0.0             0.0   
2076               0.032258           0.967742            0.0             0.0   
2077               0.032258           0.967742            0.0             0.0   

      Freeze Count  Severe Storm Count  ...  Attraction_Sling Shot  \
0              0.0            0.166667  ...                      0   
1              0.0            0.166667  ...                      0   
2              0.0            0.166667  ...                      0   
3              0.0            0.166667  ...                      0   
4              0.0            0.166667  ...                      0   
...            ...                 ...  ...                    ...   
2073           0.0            0.166667  ...                      0   
2074           0.0            0.166667  ...                      0   
2075           0.0            0.166667  ...                      0   
2076           0.0            0.166667  ...                      0   
2077           0.0            0.166667  ...                      0   

      Attraction_Spinning Coaster  Attraction_Spiral Slide  \
0                               0                        0   
1                               0                        0   
2                               0                        0   
3                               0                        0   
4                               0                        0   
...                           ...                      ...   
2073                            0                        0   
2074                            0                        0   
2075                            0                        0   
2076                            0                        0   
2077                            0                        0   

      Attraction_Superman Ride  Attraction_Swing Ride  \
0                            0                      0   
1                            0                      0   
2                            0                      0   
3                            0                      0   
4                            0                      0   
...                        ...                    ...   
2073                         0                      0   
2074                         0                      0

In [350]:
df_final.columns.unique()

Index(['Year_Month', 'Max_wait_time', 'Total_units',
       'Park_Fatalities_Previous_Year', 'Fraction of Not Rainy',
       'Fraction of Rainy', 'Drought Count', 'Flooding Count', 'Freeze Count',
       'Severe Storm Count', 'Tropical Cyclone Count', 'Wildfire Count',
       'Winter Storm Count', 'All Disasters Count', 'Public_Holidays_in_Month',
       'Spring', 'Summer', 'Autumn', 'Winter', 'Ongoing Disasters',
       'Avg_no_parades', 'Average_attendee_seaworld', 'Attraction_Bumper Cars',
       'Attraction_Bungee Jump', 'Attraction_Circus Train',
       'Attraction_Crazy Bus', 'Attraction_Crazy Dance',
       'Attraction_Dizzy Dropper', 'Attraction_Drop Tower',
       'Attraction_Flying Coaster', 'Attraction_Free Fall',
       'Attraction_Giant Wheel', 'Attraction_Giga Coaster',
       'Attraction_Go-Karts', 'Attraction_Gondola', 'Attraction_Haunted House',
       'Attraction_Himalaya Ride', 'Attraction_Inverted Coaster',
       'Attraction_Kiddie Coaster', 'Attraction_Log Flume',